### This demo shows how SGWR model can be used for prediction purposes

In [ ]:
from sgwr import ALPHA, SGWR
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

data = pd.read_csv(r"\Crime.csv")

data = np.array(data)

x_x = data[:, 0]   # longitude
y_y = data[:, 1]   # latitude
g_y = data[:, 2].reshape(-1, 1)  # dependent variable
g_x = data[:, 3:]  # independent variables

# ------------------------------------------------------------------
# Train-test split (use last 200 rows for prediction)
# ------------------------------------------------------------------
n = data.shape[0]
n_pred = 300
n_train = n - n_pred

coords_train = np.column_stack((x_x[:n_train], y_y[:n_train]))
coords_pred  = np.column_stack((x_x[n_train:], y_y[n_train:]))

X_train = g_x[:n_train]
y_train = g_y[:n_train]

X_pred = g_x[n_train:]
y_true = g_y[n_train:]  # actual values for evaluation

# ------------------------------------------------------------------
# Standardize predictors (optional but recommended)
# ------------------------------------------------------------------
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)

X_pred = scaler_x.transform(X_pred)  # use same scaler!
y_true_std = scaler_y.transform(y_true)

# ------------------------------------------------------------------
# Train SGWR model
# ------------------------------------------------------------------
selector = ALPHA(coords_train, y_train, X_train, pd.DataFrame(X_train))
bw, alpha = selector.fit()
print('Optimized alpha:', alpha)
print('BW:', bw)
sgwr_model = SGWR(coords_train, y_train, X_train, bw, pd.DataFrame(X_train), alpha)
result = sgwr_model.fit()

print('Training R²:', result.R2)
print('Adjusted R²:', result.adj_R2)
print('AICc:', result.aicc)
print('BIC:', result.bic)

# ------------------------------------------------------------------
# Predict on the 200 unseen locations
# ------------------------------------------------------------------
pred_results = sgwr_model.predict(coords_pred, X_pred)
y_pred_std = pred_results.predictions

# Inverse-transform predictions to original scale
y_pred = scaler_y.inverse_transform(y_pred_std)
y_true = scaler_y.inverse_transform(y_true_std)

# ------------------------------------------------------------------
# Evaluate predictive accuracy
# ------------------------------------------------------------------
mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Predictive MAE:", mae)
print("Predictive RMSE:", rmse)